In [ ]:
# Import Module
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Visualize Time series plot

## Upload Data

In [ ]:
# cow data
file_paths = 'processed_cow1_edit.xlsx'

# complete date for cow
complete_dates = '2020-08-15'

In [ ]:
# Define behavior colors
color_versions = {
    "original1": {
        'lying': 'lightskyblue',
        'standing': 'lawngreen',
        'eating': 'gold',
        'walking': 'darkorange'
    }
}

### Fig1 Raw data : accX, accY, accZ

In [ ]:
# Determine common Y-axis ranges
ymin_acc, ymax_acc = float('inf'), float('-inf')

for path, best_date in zip(file_paths, complete_dates):
    df = pd.read_excel(path)
    df['Time'] = pd.to_datetime(df['Time'])
    df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()]
    if not df.empty:
        for col in ['accX', 'accY', 'accZ']:
            ymin_acc = min(ymin_acc, df[col].min())
            ymax_acc = max(ymax_acc, df[col].max())

# Plotting
for version_name, behavior_colors in color_versions.items():
    fig, axs = plt.subplots(1, 1, figsize=(16, 20), sharex=True)

    for i, (path, best_date) in enumerate(zip(file_paths, complete_dates)):
        df = pd.read_excel(path)
        df['Time'] = pd.to_datetime(df['Time'])
        df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()].copy()

        df['Hour'] = df['Time'].dt.hour + df['Time'].dt.minute / 60
        df['Behavior_Change'] = (df['Classification'] != df['Classification'].shift()).cumsum()

        ax = axs[i]

        # accX, accY, accZ
        ax.plot(df['Hour'], df['accX'], color='red', label='accX')
        ax.plot(df['Hour'], df['accZ'], color='blue', label='accZ')
        ax.plot(df['Hour'], df['accY'], color='green', label='accY')

        # Y-axis range
        ax.set_ylim(ymin_acc, ymax_acc)
        ax.set_ylabel('Acceleration (m/s²)', fontsize=14)
        ax.set_title(f'Cow {i+1}', fontsize=16)
        ax.tick_params(axis='both', labelsize=12)
        ax.grid(True)

        for _, segment in df.groupby('Behavior_Change'):
            behavior = segment['Classification'].iloc[0]
            ax.axvspan(segment['Hour'].iloc[0], segment['Hour'].iloc[-1],
                       color=behavior_colors.get(behavior, 'grey'), alpha=0.6)

        ax.set_xticks([0, 3, 6, 9, 12, 15, 18, 21, 24])
        ax.set_xticklabels(['00:00', '03:00', '06:00', '09:00', '12:00',
                            '15:00', '18:00', '21:00', '24:00'])

    axs[-1].set_xlabel("Hour of Day", fontsize=11)

    handles = [
        plt.Line2D([0], [0], color='red', lw=2, label='accX'),
        plt.Line2D([0], [0], color='green', lw=2, label='accY'),
        plt.Line2D([0], [0], color='blue', lw=2, label='accZ')
    ] + [
        plt.Line2D([0], [0], linestyle='none', marker='s', markersize=10,
                   markerfacecolor=color, label=behavior.capitalize())
        for behavior, color in behavior_colors.items()
    ]

    fig.legend(handles=handles, loc='lower center', ncol=8, frameon=True,
               fontsize=14, bbox_to_anchor=(0.5, -0.0001))

    plt.suptitle(f"Acceleration Patterns (accX, accY, accZ) - {version_name}", fontsize=15)
    plt.tight_layout(rect=[0, 0.04, 1, 0.95])
    plt.show()

### Fig2 Raw data : gyroX, gyroY, gyroZ

In [ ]:
# Determine common Y-axis range
ymin_gyro, ymax_gyro = float('inf'), float('-inf')

for path, best_date in zip(file_paths, complete_dates):
    df = pd.read_excel(path)
    df['Time'] = pd.to_datetime(df['Time'])
    df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()]
    if not df.empty:
        for col in ['gyroX', 'gyroY', 'gyroZ']:
            ymin_gyro = min(ymin_gyro, df[col].min())
            ymax_gyro = max(ymax_gyro, df[col].max())

# Plotting
for version_name, behavior_colors in color_versions.items():
    fig, axs = plt.subplots(1, 1, figsize=(16, 20), sharex=True)

    for i, (path, best_date) in enumerate(zip(file_paths, complete_dates)):
        df = pd.read_excel(path)
        df['Time'] = pd.to_datetime(df['Time'])
        df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()].copy()

        df['Hour'] = df['Time'].dt.hour + df['Time'].dt.minute / 60
        df['Behavior_Change'] = (df['Classification'] != df['Classification'].shift()).cumsum()

        ax = axs[i]

        # Plot gyroX, gyroY, gyroZ
        ax.plot(df['Hour'], df['gyroY'], color='red', label='gyroY')
        ax.plot(df['Hour'], df['gyroZ'], color='blue', label='gyroZ')
        ax.plot(df['Hour'], df['gyroX'], color='green', label='gyroX')

        # Set consistent Y-axis across all cows
        ax.set_ylim(ymin_gyro, ymax_gyro)
        ax.set_ylabel('Angular Rotation (°/s)', fontsize=14)
        ax.set_title(f'Cow {i+1}', fontsize=16)
        ax.tick_params(axis='both', labelsize=12)
        ax.grid(True)

        # Behavior shading
        for _, segment in df.groupby('Behavior_Change'):
            behavior = segment['Classification'].iloc[0]
            ax.axvspan(segment['Hour'].iloc[0], segment['Hour'].iloc[-1],
                       color=behavior_colors.get(behavior, 'grey'), alpha=0.5)

        ax.set_xticks([0, 3, 6, 9, 12, 15, 18, 21, 24])
        ax.set_xticklabels(['00:00', '03:00', '06:00', '09:00', '12:00',
                            '15:00', '18:00', '21:00', '24:00'])

    axs[-1].set_xlabel("Hour of Day", fontsize=11)

    # Legend setup
    handles = [
        plt.Line2D([0], [0], color='green', lw=2, label='gyroX'),
        plt.Line2D([0], [0], color='red', lw=2, label='gyroY'),
        plt.Line2D([0], [0], color='blue', lw=2, label='gyroZ')
    ] + [
        plt.Line2D([0], [0], linestyle='none', marker='s', markersize=10,
                   markerfacecolor=color, label=behavior.capitalize())
        for behavior, color in behavior_colors.items()
    ]

    fig.legend(handles=handles, loc='lower center', ncol=8,
               frameon=True, fontsize=14, bbox_to_anchor=(0.5, -0.0001))

    plt.suptitle(f"Gyroscope Patterns (gyroX, gyroY, gyroZ) - {version_name}", fontsize=15)
    plt.tight_layout(rect=[0, 0.04, 1, 0.95])
    plt.show()

### Fig3 Signal Vector Magnitude of Accelerometer : SVM_Acc

In [ ]:
# Determine global Y-axis limits
ymin_global, ymax_global = float('inf'), float('-inf')

for path, best_date in zip(file_paths, complete_dates):
    df = pd.read_excel(path)
    df['Time'] = pd.to_datetime(df['Time'])
    df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()].copy()

    df['SVM_acc_smoothed'] = df['SVM_acc'].rolling(window=30, min_periods=1).mean()
    
    current_min = df['SVM_acc_smoothed'].min()
    current_max = df['SVM_acc_smoothed'].max()
    
    ymin_global = min(ymin_global, current_min)
    ymax_global = max(ymax_global, current_max)

# Plotting 
for version_name, behavior_colors in color_versions.items():
    fig, axs = plt.subplots(1, 1, figsize=(16, 20), sharex=True)

    for i, (path, best_date) in enumerate(zip(file_paths, complete_dates)):
        df = pd.read_excel(path)
        df['Time'] = pd.to_datetime(df['Time'])
        df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()].copy()

        df['Hour'] = df['Time'].dt.hour + df['Time'].dt.minute / 60
        df['Behavior_Change'] = (df['Classification'] != df['Classification'].shift()).cumsum()
        df['SVM_acc_smoothed'] = df['SVM_acc'].rolling(window=30, min_periods=1).mean()

        ax = axs[i]
        ax.plot(df['Hour'], df['SVM_acc_smoothed'], color='magenta', label='SVM_acc (smoothed)')

        ax.set_ylim(ymin_global, ymax_global)
        ax.set_ylabel('SVM_Acc (m/s²)', fontsize=14)
        ax.set_title(f'Cow {i+1}', fontsize=16)
        ax.tick_params(axis='both', labelsize=12)
        ax.grid(True)

        for _, segment in df.groupby('Behavior_Change'):
            behavior = segment['Classification'].iloc[0]
            ax.axvspan(segment['Hour'].iloc[0], segment['Hour'].iloc[-1],
                       color=behavior_colors.get(behavior, 'grey'), alpha=0.6)

        ax.set_xticks([0, 3, 6, 9, 12, 15, 18, 21, 24])
        ax.set_xticklabels(['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00', '24:00'])

    axs[-1].set_xlabel("Hour of Day", fontsize=11)

    handles = [
        plt.Line2D([0], [0], color='magenta', lw=2, label='SVM_acc')
    ] + [
        plt.Line2D([0], [0], linestyle='none', marker='s', markersize=10,
                   markerfacecolor=color, label=behavior.capitalize())
        for behavior, color in behavior_colors.items()
    ]

    fig.legend(handles=handles, loc='lower center', ncol=5, frameon=True, fontsize=14, bbox_to_anchor=(0.5, -0.0001))
    plt.suptitle(f"SVM_acc Patterns - {version_name} Palette", fontsize=15)
    plt.tight_layout(rect=[0, 0.04, 1, 0.95])
    plt.show()

### Fig4 Signal Vector Magnitude of Gyroscope: SVM_Gyro

In [ ]:
# Determine global Y-axis 
ymin_global, ymax_global = float('inf'), float('-inf')

for path, best_date in zip(file_paths, complete_dates):
    df = pd.read_excel(path)
    df['Time'] = pd.to_datetime(df['Time'])
    df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()].copy()
    df['SVM_gyro_smoothed'] = df['SVM_gyro'].rolling(window=30, min_periods=1).mean()
    
    ymin_global = min(ymin_global, df['SVM_gyro_smoothed'].min())
    ymax_global = max(ymax_global, df['SVM_gyro_smoothed'].max())

# Plotting
for version_name, behavior_colors in color_versions.items():
    fig, axs = plt.subplots(1, 1, figsize=(16, 20), sharex=True)

    for i, (path, best_date) in enumerate(zip(file_paths, complete_dates)):
        df = pd.read_excel(path)
        df['Time'] = pd.to_datetime(df['Time'])
        df = df[df['Time'].dt.date == pd.to_datetime(best_date).date()].copy()

        df['Hour'] = df['Time'].dt.hour + df['Time'].dt.minute / 60
        df['Behavior_Change'] = (df['Classification'] != df['Classification'].shift()).cumsum()
        df['SVM_gyro_smoothed'] = df['SVM_gyro'].rolling(window=30, min_periods=1).mean()

        ax = axs[i]
        ax.plot(df['Hour'], df['SVM_gyro_smoothed'], color='orangered', label='SVM_gyro (smoothed)')
        ax.set_ylim(ymin_global, ymax_global)

        ax.set_ylabel('SVM_Gyro (°/s)', fontsize=14)
        ax.set_title(f'Cow {i+1}', fontsize=16)
        ax.tick_params(axis='both', labelsize=12)
        ax.grid(True)

        for _, segment in df.groupby('Behavior_Change'):
            behavior = segment['Classification'].iloc[0]
            ax.axvspan(segment['Hour'].iloc[0], segment['Hour'].iloc[-1],
                       color=behavior_colors.get(behavior, 'grey'), alpha=0.6)

        ax.set_xticks([0, 3, 6, 9, 12, 15, 18, 21, 24])
        ax.set_xticklabels(['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00', '24:00'])

    axs[-1].set_xlabel("Hour of Day", fontsize=11)

    handles = [
        plt.Line2D([0], [0], color='orangered', lw=2, label='SVM_gyro (smoothed)')
    ] + [
        plt.Line2D([0], [0], linestyle='none', marker='s', markersize=10,
                   markerfacecolor=color, label=behavior.capitalize())
        for behavior, color in behavior_colors.items()
    ]

    fig.legend(handles=handles, loc='lower center', ncol=5, frameon=True, fontsize=14, bbox_to_anchor=(0.5, -0.0001))
    plt.suptitle(f"SVM_gyro Patterns{version_name} Palette", fontsize=15)
    plt.tight_layout(rect=[0, 0.04, 1, 0.95])
    plt.show()